In [1]:
# from __future__ import unicode_literals
import time, json, requests
import time
import json
import requests
from datetime import datetime
import numpy as np
import matplotlib
import matplotlib.figure
from matplotlib.font_manager import FontProperties
from matplotlib.backends.backend_agg import FigureCanvasAgg
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
##pip3 install --user git+https://github.com/matplotlib/basemap.git
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

## Raw Data retrieved from Tencent Realtime NCP report page

In [75]:
url='https://view.inews.qq.com/g2/getOnsInfo?name=disease_h5&&callback=&_=%d'%int(time.time()*1000)

rawData = json.loads(requests.get(url=url).json()['data'])

In [82]:
with open("./raw_data.json","w") as f:
    json.dump(rawData,f,indent=4,ensure_ascii=False)

In [6]:
def retrieve_current_detailed_data(rawData):
    
    processedData = {}
    for countryDict_ in rawData["areaTree"]:
        country = countryDict_["name"]
        country_today = countryDict_["today"]
        country_total = countryDict_["total"]
        processedData[country] = {"today" : country_today, "total" :country_total,"province" : {},}
        
        if "children" in countryDict_:
            for provinceDict_ in countryDict_["children"]:
                province = provinceDict_["name"]
                province_today = provinceDict_["today"]
                province_total = provinceDict_["total"]
                processedData[country]["province"][province] = {"today" : province_today,"total" : province_total,"district" : {},}
                if "children" in provinceDict_:
                    for districtDict_ in provinceDict_["children"]:
                        district = districtDict_["name"]
                        district_today = districtDict_["today"]
                        district_total = districtDict_["total"]
                        processedData[country]["province"][province]["district"][district] = {"today" : province_today,"total" : province_total,}
    return processedData
                        

In [8]:
print(json.dumps(retrieve_current_detailed_data(rawData),indent=4,ensure_ascii=False))


{
    "中国": {
        "today": {
            "confirm": 2055,
            "suspect": 0,
            "dead": 0,
            "heal": 0,
            "isUpdated": true
        },
        "total": {
            "confirm": 44763,
            "suspect": 16067,
            "dead": 1115,
            "heal": 4795,
            "showRate": false,
            "showHeal": true,
            "deadRate": 2.49,
            "healRate": 10.71
        },
        "province": {
            "湖北": {
                "today": {
                    "confirm": 1638,
                    "suspect": 0,
                    "dead": 0,
                    "heal": 0,
                    "isUpdated": true
                },
                "total": {
                    "confirm": 33366,
                    "suspect": 0,
                    "dead": 1068,
                    "heal": 2639,
                    "showRate": true,
                    "showHeal": true,
                    "deadRate": 3.2,
                    "he

In [73]:
def plot_distribution(processedData):
    
    ChinaData = processedData["中国"]["province"]
    
    font = FontProperties(fname='china-shapefiles/Sansation-Regular.ttf', size=12)
    lat_min = 0
    lat_max = 60
    lon_min = 70
    lon_max = 140
    
    handles = [
            matplotlib.patches.Patch(color='#ffaa85', alpha=1, linewidth=0),
            matplotlib.patches.Patch(color='#ff7b69', alpha=1, linewidth=0),
            matplotlib.patches.Patch(color='#bf2121', alpha=1, linewidth=0),
            matplotlib.patches.Patch(color='#7f1818', alpha=1, linewidth=0),
]
    labels = [ '1-9 ppl', '10-99 ppl', '100-999 ppl', '>1000 ppl']
    
    fig = matplotlib.figure.Figure()
    fig.set_size_inches(10, 10) 
    axes = fig.add_axes((0.1, 0.12, 0.8, 0.8))
    
    m = Basemap(llcrnrlon=lon_min, urcrnrlon=lon_max, llcrnrlat=lat_min, urcrnrlat=lat_max, resolution='l', ax=axes)
#     m = Basemap(projection='ortho', lat_0=30, lon_0=105, resolution='l', ax=axes)

    m.readshapefile('./china-shapefiles/china', 'province', drawbounds=True)
    m.readshapefile('./china-shapefiles/china_nine_dotted_line', 'section', drawbounds=True)
    m.drawcoastlines(color='black') 
    m.drawcountries(color='black') 
    m.drawparallels(np.arange(lat_min,lat_max,10), labels=[1,0,0,0]) #latitude
    m.drawmeridians(np.arange(lon_min,lon_max,10), labels=[0,0,0,1]) #longitute
    
    for info, shape in zip(m.province_info, m.province):
        pname = info['OWNER'].strip('\x00')
        fcname = info['FCNAME'].strip('\x00')
        if pname != fcname:
            continue
        
        for key in ChinaData.keys():
            if key in pname:
                if ChinaData[key]["total"]["confirm"] == 0:
                    color = '#f0f0f0'
                    poly = Polygon(shape, facecolor=color, edgecolor=color)
                    axes.add_patch(poly)
                elif ChinaData[key]["total"]["confirm"] < 10:
                    color = '#ffaa85'
                    poly = Polygon(shape, facecolor=color, edgecolor=color)
                    axes.add_patch(poly)
                elif ChinaData[key]["total"]["confirm"] < 100:
                    color = '#ff7b69'
                    poly = Polygon(shape, facecolor=color, edgecolor=color)
                    axes.add_patch(poly)
                elif ChinaData[key]["total"]["confirm"] < 1000:
                    color = '#bf2121'
                    poly = Polygon(shape, facecolor=color, edgecolor=color)
                    axes.add_patch(poly)
                else:
                    color = '#7f1818'
                    poly = Polygon(shape, facecolor=color, edgecolor=color)
                    axes.add_patch(poly)
                break
                


    
    axes.legend(handles, labels, bbox_to_anchor=(0.5, -0.11), loc='lower center', ncol=4, prop=font)
    axes.set_title("2019-nCoV distribution map", fontproperties=font)
    FigureCanvasAgg(fig)
    fig.savefig('2019-nCoV疫情地图.png')
    fig.set_visible(b=True)

In [74]:
plot_distribution(retrieve_current_detailed_data(rawData))